In [235]:
import requests #pip install requests
from bs4 import BeautifulSoup #pip install bs4
import os
import time
import json
import re
import pandas as pd

API's used:
- Beautiful Soup: https://www.crummy.com/software/BeautifulSoup/bs4/doc/
- Requests : https://requests.readthedocs.io/en/master/

Futher Work:
- Database: https://medium.com/analytics-vidhya/tutorial-amazon-price-tracker-using-python-and-mongodb-part-1-aece6347ec63
- Proxy Servers, Exceptions, crawlers : https://blog.hartleybrody.com/scrape-amazon/

In [250]:
url = "https://www.ebay.com/itm/Reebok-Mens-Astroride-Strike-Shoes/174184909994?_trkparms=aid%3D1110006%26algo%3DHOMESPLICE.SIM%26ao%3D1%26asc%3D20200520130048%26meid%3D6c918a478fad4c49b80b077e90359402%26pid%3D100005%26rk%3D3%26rkt%3D12%26mehot%3Dco%26sd%3D174231781782%26itm%3D174184909994%26pmt%3D1%26noa%3D0%26pg%3D2047675%26algv%3DSimplAMLv5PairwiseWebWithBBEV2bDemotion%26brand%3DReebok&_trksid=p2047675.c100005.m1851"

In [253]:
def ParseHTML(url, printHTML = False):
    r = requests.get(url, headers=headers)
        
    if r.status_code != 200:        
        print("Page %s must have been blocked by eBay as the status code was %d"%(url,r.status_code))
        return None
    
    print("Successfully parsed page")
    soup = BeautifulSoup(r.content, features="lxml")
    
    if (printHTML):
        print(soup)
    
    return soup

In [256]:
html = ParseHTML(url)

Successfully parsed page


In [221]:
def GetProductName_eBay(html):
    nameTag = str(html.find(id='itemTitle'))
    name = re.search("</span>(.*)</h1>", nameTag).group(1)    
    return name

In [222]:
GetProductName_eBay(html)

"Reebok Men's Astroride Strike Shoes"

In [223]:
def GetProductPrice_eBay(html):
    priceTag = str(html.find(id='prcIsum'))
    price = float(re.search("\$(.*)</span>", priceTag).group(1))
    return price

In [224]:
GetProductPrice_eBay(html)

26.31

In [225]:
def GetSimiliarProducts_ebay(html):
    suggestionsHtml = html.find("ul", attrs={"class": ['mfe-recos', 'mfe-recos-slider', 'clear', 'limit-small-screen']}).find_all("li")
    suggestionsDict = {}

    for suggestion in suggestionsHtml:
        url = suggestion.find("a")['href']
        name = suggestion.find("p").text
        suggestionsDict[name] = url

    return suggestionsDict

In [226]:
GetSimiliarProducts_ebay(html)

{"Reebok Fulgere Men's Slide Shoes": 'https://www.ebay.com/itm/Reebok-Fulgere-Mens-Slide-Shoes/174231781722?_trkparms=aid%3D1110001%26algo%3DSPLICE.SIM%26ao%3D1%26asc%3D225086%26meid%3Ddc75731264a34065b9a3b68e4e5628d1%26pid%3D100623%26rk%3D1%26rkt%3D5%26mehot%3Dpp%26sd%3D174184909994%26itm%3D174231781722%26pmt%3D1%26noa%3D1%26pg%3D2047675&_trksid=p2047675.c100623.m-1',
 "Reebok Lite 2 Men's Shoes": 'https://www.ebay.com/itm/Reebok-Lite-2-Mens-Shoes/174231781789?_trkparms=aid%3D1110001%26algo%3DSPLICE.SIM%26ao%3D1%26asc%3D225086%26meid%3Ddc75731264a34065b9a3b68e4e5628d1%26pid%3D100623%26rk%3D2%26rkt%3D5%26mehot%3Dpp%26sd%3D174184909994%26itm%3D174231781789%26pmt%3D1%26noa%3D1%26pg%3D2047675&_trksid=p2047675.c100623.m-1',
 "Men's New Outdoor Sneakers Breathable Casual Sports Athletic Running Shoes": 'https://www.ebay.com/itm/Mens-New-Outdoor-Sneakers-Breathable-Casual-Sports-Athletic-Running-Shoes/153971434367?_trkparms=aid%3D1110001%26algo%3DSPLICE.SIM%26ao%3D1%26asc%3D225086%26meid%3Dd

In [248]:
def MonitorPrice(url, maximumPrice = 40, maximumTime = 10):
    
    html = ParseHTML(url)
    productPrice = GetProductPrice_eBay(html)
    productName = GetProductName_eBay(html)
    
    if (productPrice < maximumPrice):
        return " GO BUY"

    print("Monioting product: " + productName + " \n")
    startTime = time.time()
    
    while ( productPrice > maximumPrice and time.time()-startTime < maximumTime):

        productPrice = GetProductPrice_eBay(html)
        
        print(" Time: " + str(time.ctime()) )
        print(" Price: " + str(productPrice) + " \n")
        
        if (productPrice < maximumPrice):
            return "GO BUY!!!"
        
        time.sleep(3)

        
    similiarProducts = GetSimiliarProducts_ebay(html)
    print("Product price did not drop to required value in given time frame. \n")
    print("Some similiar products:")
    
    for item in similiarProducts.keys():
        print(item)
        print(similiarProducts[item])
        print()
    
    return "Sorry"

In [249]:
MonitorPrice(url, 20, 15)

Successfully parsed page
Monioting product: Reebok Men's Astroride Strike Shoes 

 Time: Fri Jul 17 18:14:17 2020
 Price: 26.31 

 Time: Fri Jul 17 18:14:20 2020
 Price: 26.31 

 Time: Fri Jul 17 18:14:23 2020
 Price: 26.31 

 Time: Fri Jul 17 18:14:26 2020
 Price: 26.31 

 Time: Fri Jul 17 18:14:29 2020
 Price: 26.31 

Product price did not drop to required value in given time frame. 

Some similiar products:
Reebok Lite 2 Men's Shoes
https://www.ebay.com/itm/Reebok-Lite-2-Mens-Shoes/174231781789?_trkparms=aid%3D1110001%26algo%3DSPLICE.SIM%26ao%3D1%26asc%3D20160323102634%26meid%3Df701515bb9684ddc88a9dff0027f71b9%26pid%3D100623%26rk%3D1%26rkt%3D5%26mehot%3Dpp%26sd%3D174184909994%26itm%3D174231781789%26pmt%3D1%26noa%3D1%26pg%3D2047675&_trksid=p2047675.c100623.m-1

Reebok Fulgere Men's Slide Shoes
https://www.ebay.com/itm/Reebok-Fulgere-Mens-Slide-Shoes/174231781722?_trkparms=aid%3D1110001%26algo%3DSPLICE.SIM%26ao%3D1%26asc%3D20160323102634%26meid%3Df701515bb9684ddc88a9dff0027f71b9%26pid

'Sorry'